<div style="background-color: #ADD8E6; border: 1px solid gray; padding: 3px">
    This notebook consists of 2 agentic workflows:
        <h3>Data Generation Workflow</h3>
        <li><b>Data Augmentation</b>: Augments the provided image dataset.</li>
        <h3>Validation Workflow</h3>
        <li><b>Image Validator</b>: Identifies whether a valid driver's license exists in the given image.</li>
        <li><b>Data Extractor</b>: Extracts relevant metadata from the image.</li>
        <li><b>Application Validator</b>: Given the extracted metadata associated with the application, uses a set of predefined rules to validate the driver's license application.</li>
</div>

In [4]:
##############################################################################
# Imports
##############################################################################
# import pysqlite3 as sqlite3
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import operator
from typing import Annotated, TypedDict, List, Optional, Literal
from langgraph.graph import StateGraph, END, START
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from PIL import Image
import pytesseract
import io
import json
from datetime import datetime
import re
import os
import requests
from io import BytesIO
from dotenv import load_dotenv
import mimetypes
import base64
from urllib.parse import urlparse
from PIL import Image
from io import BytesIO
import requests
load_dotenv()
import traceback
from openai import OpenAI
import instructor
from pydantic import BaseModel, Field, TypeAdapter
from more_itertools import chunked
import utils

In [5]:
##############################################################################
# LLMs
##############################################################################

vision_llm = instructor.from_openai(OpenAI(
        
    api_key=os.getenv('LLAMASCOUT4_LLM_KEY'),
    
    base_url=os.getenv('LLAMASCOUT4_LLM_BASE'),
))

In [6]:
##############################################################################
# Structured Output
##############################################################################

class DriversLicenseField(BaseModel):
    
    value: str = Field("", description="Name of field")
    
    missing_error_reason: str = Field("", description="Reason for missing field")

    is_valid: Optional[bool] = Field(None, description="Indicates whether the license is valid.")
    
    application_value: str = Field("", description="Value of the corresponding field in the application")
    
    invalid_error_reason: str = Field("", description="Reason for invalid field")

    

class DriversLicenseMetadata(BaseModel):

    application_id: str = Field("", description="Unique identifier")

    model: str = Field("", description="Name of LLM used to generate metadata")
    
    name: DriversLicenseField = Field(description="Name of driver's license owner")
    
    date_of_birth: DriversLicenseField = Field(description="Date of birth of driver's license owner")
    
    expiration_date: DriversLicenseField = Field(description="Expiration date of driver's license")
    
    state_issued: DriversLicenseField = Field(description="State where the license was issued")
    
    issuance_date: DriversLicenseField = Field(description="Date when the license was issued")

    photo_orientation: DriversLicenseField = Field(description="The skew of the license in the photo")

class LicenseApplication(BaseModel):

    application_id: str = Field(description="Unique identifier")

    image_path: str = Field(description="Image path")

    application_data: dict = Field(description="Submitted application data")
    

In [4]:
##############################################################################
# Tools
##############################################################################

def image_to_base64(image_path, encode_image_bytes=False):
    """Transforms image at provided local path or URL into base64-encoded representation."""
    
    def is_valid_http_url(input_path):
        """Returns whether or not the input is a valid URL."""
    
        parsed_url = urlparse(input_path)
    
        is_http_url = all([parsed_url.scheme in ('http', 'https'), parsed_url.netloc])
    
        return is_http_url
        
    
    try:
        start_time = datetime.now()
    
        mime_type, _ = mimetypes.guess_type(image_path)
    
        if "image" in mime_type:
    
            if encode_image_bytes:
    
                if is_valid_http_url(image_path):
        
                    response = requests.get(image_path)
    
                    response.raise_for_status()
    
                    stream_to_read = response.content
    
                else:
                    
                    with open(image_path, "rb") as image_file:
                        
                        stream_to_read = image_file.read()
    
                img = base64.b64encode(stream_to_read).decode("utf-8")
        
                return f"data:{mime_type};base64,{img}"
    
            else:
    
                return image_path
                
    
        else:
    
            raise Exception(f"Mime type {mime_type} not supported")
        
        processing_time = (datetime.now() - start_time).total_seconds()
        
        print(f"Image loaded: time: {processing_time:.2f}s")
            
    except Exception as e:
        
        error_msg = f"Image loading error: {str(e)}"
    
        print(f"- {error_msg}")

        traceback.print_exc()
    
    return None

In [5]:
##############################################################################
# Prompts
##############################################################################
content_extractor_system_prompt = """You are an expert at extracting information from U.S. driver's licenses.

Given the above image and context, extract the following data from the image. If you cannot extract the specified data, say that the field or characteristic was missing or unclear.

Return JSON:
{{
  "name": {
    "value": "The driver license owner name",
    "missing_error_reason": "Explanation of missing name, if applicable, or blank if the driver license owner is present"
   },
   "date_of_birth": {
    "value": "The date of birth of the driver's license owner",
    "missing_error_reason": "Explanation of missing date of birth, if applicable, or blank if the date of birth is present"
   },
   "expiration_date": {
    "value": "The expiration date of the driver's license",
    "missing_error_reason": "Explanation of missing expiration date, if applicable, or blank if the expiration date is present"
   },
   "issuance_date": {
    "value": "The issuance date of the driver's license",
    "missing_error_reason": "Explanation of missing issuance date, if applicable, or blank if the issuance date is present"
   },
   "state_issued": {
    "value": "The driver license state",
    "missing_error_reason": "Explanation of missing state, if applicable, or blank if the state is present"
   },
   "photo_orientation": {
    "value": "Whether or not the license in the image is skewed",
    "missing_error_reason": "Explanation if the skew could not be detected, or blank if the skew could be detected"
   },
}}
"""

content_extractor_human_prompt = """
"Image:\n\n{image}\n\nExtracted data:
"""

image_validation_system_prompt = """
You are a data quality expert. Validate this driver's license data.

Return JSON:
{{
  "is_valid": true/false,
  "completeness_score": 0-100,
  "confidence_score": 0-100,
  "critical_issues": ["list critical problems"],
  "warnings": ["list minor issues"],
  "missing_fields": ["list missing required fields"],
  "recommendations": ["improvement suggestions"]
}}"""

image_validation_human_prompt = """You are a data quality expert. Validate this driver's license data.

Check for:
1. Critical fields present (license_number, name, date_of_birth)
2. Date format correctness and logical consistency
3. State codes are valid (2 letters)
4. ZIP codes are 5 digits
5. Gender codes are valid (M, F, X)
6. Expiration date is after issue date
7. Issue date is not in the future
8. Person is old enough to drive (check date_of_birth)

Return JSON:
{{
  "is_valid": true/false,
  "completeness_score": 0-100,
  "confidence_score": 0-100,
  "critical_issues": ["list critical problems"],
  "warnings": ["list minor issues"],
  "missing_fields": ["list missing required fields"],
  "recommendations": ["improvement suggestions"]
}}"""

In [6]:
##############################################################################
# Test
##############################################################################
image_path = "https://raw.githubusercontent.com/agapebondservant/dla_poc/refs/heads/main/notebooks/data2/DENVER-25CAP-00000-04SVL-ID.jpeg"

base64_data_url = image_to_base64(image_path, encode_image_bytes=False)

if base64_data_url:
    
    response = vision_llm.chat.completions.create(
    
        model=os.getenv('LLAMASCOUT4_LLM_NAME'),
        
        messages=[
            {
                "role": "system",
                "content": [
                    {"type": "text", "text": content_extractor_system_prompt},
                ]
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": content_extractor_human_prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": base64_data_url, "detail": "high"}, 
                    },
                ],
            }
        ],

        response_model=DriversLicenseMetadata,
        
        max_tokens=8192,

        temperature=0,
    )
    print(response)

model='' name=DriversLicenseField(value='William Blast', error_reason='', is_valid=None) date_of_birth=DriversLicenseField(value='07/18/1967', error_reason='', is_valid=None) expiration_date=DriversLicenseField(value='07/18/2029', error_reason='', is_valid=None) state_issued=DriversLicenseField(value='Colorado', error_reason='', is_valid=None) issuance_date=DriversLicenseField(value='12/10/2024', error_reason='', is_valid=None) photo_orientation=DriversLicenseField(value='Not skewed', error_reason='', is_valid=None)


In [8]:
# completion_content = response.choices[0].message.content
# completion_content.replace("```json", "").replace("```", "")
# content = json.loads(response.choices[0].message.content)
# metadata = DriversLicenseMetadata.model_validate(content)
# metadata.name.field
# content
response.name

DriversLicenseField(value='William Blast', error_reason='', is_valid=None)

In [2]:
##############################################################################
# State Definitions
##############################################################################

class LicenseState(TypedDict):
    """Enhanced state definition"""
    image_applications: List[LicenseApplication]
    
    image_data: Annotated[List[DriversLicenseMetadata], operator.add]
    
    messages: Annotated[List[BaseMessage], operator.add]

In [3]:
##############################################################################
# LLMs
##############################################################################

main_llm = ChatOpenAI(
    
    model=os.getenv('LLAMASCOUT4_LLM_NAME'),
    
    api_key=os.getenv('LLAMASCOUT4_LLM_KEY'),
    
    base_url=os.getenv('LLAMASCOUT4_LLM_BASE'),

    max_tokens = 8192,

    temperature = 0
)

In [5]:
##############################################################################
# Nodes
##############################################################################

def extract_license_data_with_llm(state: LicenseState) -> LicenseState:
    """Node: Extract structured data"""
    print("✓ STEP 1: AI-Powered Data Extraction")
    
    print("="*60)
    
    llm = main_llm
    
    extraction_prompt = ChatPromptTemplate.from_messages([
        
        ("system", extract_data_system_prompt),
        
        ("human", content_extractor_human_prompt)
    ])
    
    try:
        start_time = datetime.now()
        
        chain = extraction_prompt | llm
        
        response = chain.invoke({"image": state["image_data"]})
        
        response_text = response.content if hasattr(response, 'content') else str(response)

        print(response_text)
        
        json_text = response_text
        if "```json" in response_text:
            
            json_text = response_text.split("```json")[1].split("```")[0]
            
        elif "```" in response_text:
            
            json_text = response_text.split("```")[1].split("```")[0]
        
        extracted_data = json.loads(json_text.strip())
            
        state["extracted_data"] = extracted_data
        
        processing_time = (datetime.now() - start_time).total_seconds()

        non_null_fields = sum(1 for v in extracted_data.values() if v is not None and v != "")
        
        total_fields = len(extracted_data)
        
        print(f"✅ Extraction completed")
        
        print(f"   Fields extracted: {non_null_fields}/{total_fields}")
        
        print(f"   Time: {processing_time:.2f}s")
        
        print(f"\n   Extracted data:")
        
        print(f"   {'-'*56}")
        
        for key, value in list(extracted_data.items())[:10]:
            
            if value:
                
                display_value = str(value)[:40]
                
                print(f"   {key}: {display_value}")
                
        print(f"   {'-'*56}")
        
        state["messages"].append(AIMessage(content=response_text))
        
    except Exception as e:
        error_msg = f"Extraction error: {str(e)}"
        
        state["errors"].append(error_msg)
        
        print(f"❌ {error_msg}")
        
        state["extracted_data"] = {}
    
    return state


def validate_extracted_data(state: LicenseState) -> LicenseState:
    """Node: Validate extracted data"""
    print("✓ STEP 3: Data Validation")
    
    print("="*60)
    
    llm = main_llm
    
    validation_prompt = ChatPromptTemplate.from_messages([
        ("system", image_validation_system_prompt),
        ("human", image_validation_human_prompt)
    ])
    
    try:
        start_time = datetime.now()
        
        chain = validation_prompt | llm
        response = chain.invoke({
            "data": json.dumps(state["extracted_data"], indent=2)
        })
        
        response_text = response.content if hasattr(response, 'content') else str(response)
        
        json_text = response_text
        
        if "```json" in response_text:
            
            json_text = response_text.split("```json")[1].split("```")[0]
            
        elif "```" in response_text:
            
            json_text = response_text.split("```")[1].split("```")[0]
        
        validation_result = json.loads(json_text.strip())
        
        state["validation_result"] = validation_result
        
        processing_time = (datetime.now() - start_time).total_seconds()
        
        print(f"✅ Validation completed")
        print(f"   Valid: {validation_result.get('is_valid', False)}")
        print(f"   Completeness: {validation_result.get('completeness_score', 0):.0f}%")
        print(f"   Confidence: {validation_result.get('confidence_score', 0):.0f}%")
        print(f"   Time: {processing_time:.2f}s")
        
        if validation_result.get('critical_issues'):
            
            print(f"\n   ⚠️  Critical Issues:")
            
            for issue in validation_result['critical_issues']:
                
                print(f"      • {issue}")
        
        if validation_result.get('warnings'):
            
            print(f"\n   ⚡ Warnings:")
            
            for warning in validation_result['warnings']:
                
                print(f"      • {warning}")
        
    except Exception as e:
        error_msg = f"Validation error: {str(e)}"
        
        state["errors"].append(error_msg)
        
        print(f"❌ {error_msg}")
        
        state["validation_result"] = {
            "is_valid": False,
            "completeness_score": 0,
            "confidence_score": 0,
            "critical_issues": ["Validation process failed"],
            "warnings": [],
            "missing_fields": [],
            "recommendations": ["Manual review required"]
        }
    
    return state


def compile_final_result(state: LicenseState) -> LicenseState:
    """Node: Compile final results"""
    print("✓ STEP 4: Compiling Final Results")
    
    print("="*60)
    
    has_errors = len(state["errors"]) > 0
    
    is_valid = state["validation_result"].get("is_valid", False)
    
    completeness = state["validation_result"].get("completeness_score", 0)
    
    if has_errors:
        
        status = "error"
        
    elif not is_valid or completeness < 50:
        
        status = "needs_review"
        
    elif completeness < 80:
        
        status = "partial"
        
    else:
        
        status = "success"
    
    state["result"] = {
        
        "status": status,
        
        "extracted_data": state["extracted_data"],
        
        "validation": state["validation_result"],
        
        "metadata": {
            
            "processing_timestamp": datetime.now().isoformat(),
            
            "retry_count": state.get("retry_count", 0)
        },
        "errors": state["errors"],
        
        "warnings": state["warnings"]
    }
    
    print(f"✅ Status: {status.upper()}")
    
    print(f"   Errors: {len(state['errors'])}")
    
    print(f"   Warnings: {len(state['warnings'])}")
    
    return state

In [6]:
##############################################################################
# Graph
##############################################################################
def create_license_extraction_graph():
    """Create the LangGraph workflow"""
    workflow = StateGraph(LicenseState)
    
    # Add all nodes
    workflow.add_node("load_image", load_and_preprocess_image)
    
    workflow.add_node("extract_data", extract_license_data_with_llm)
    
    # workflow.add_node("validate_data", validate_extracted_data)
    
    workflow.add_node("compile_result", compile_final_result)
    
    # Define edges
    workflow.add_edge(START, "load_image")
    
    workflow.add_edge("load_image", "extract_data")
    
    # workflow.add_edge("extract_data", "validate_data")
    
    # workflow.add_edge("validate_data", "compile_result")

    workflow.add_edge("extract_data", "compile_result")
    
    workflow.add_edge("compile_result", END)
    
    return workflow.compile()

### Execute Code Translation Flow
Execute the flow!

In [7]:
##############################################################################
# Execute the Flow
##############################################################################
paths = [
    ("https://raw.githubusercontent.com/agapebondservant/dla_poc/refs/heads/main/notebooks/data2/DENVER-25CAP-00000-04SVL-ID.jpeg", "data2/DENVER-25CAP-00000-04SVL-ID.json")
]

# applications_raw = utils.group_files_by_id(
#             "https://github.com/agapebondservant/dla_poc", 
#             "notebooks/data2")

def convert_to_license_applications(repo_url: str, repo_branch: str = "main", folder_path: str) -> list[LicenseApplication]:
    
        # Retrieve repo file data
        applications_raw = utils.group_files_by_id(
            repo_url, 
            folder_path)
        
        adapter = TypeAdapter(list[LicenseApplication])
    
        applications = adapter.validate_python(applications_raw)

        return applications
    

def extract_license_info(model: string, application: LicenseApplication) -> dict:
        """
        Main extraction function
        """
    # image_applications: List[LicenseApplication]
    
    # image_data: Annotated[List[DriversLicenseMetadata], operator.add]
    
    # messages: Annotated[List[BaseMessage], operator.add]

        start_time = datetime.now()
    
        print(f"Git Repo: {repo_url}")
    
        print(f"Started: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

        app = create_license_extraction_graph()

        # Set list of models
        vision_models = ["LLAMASCOUT4", "GEMMA27B", "GEMMA12B", "GEMMA4B"]

        # Retrieve repo file data
        applications_raw = utils.group_files_by_id(
            "https://github.com/agapebondservant/dla_poc", 
            "notebooks/data2")
        
        adapter = TypeAdapter(list[LicenseApplication])
    
        applications = adapter.validate_python(applications_raw)

        clusters = list(chunked(my_list, 3))

        for application in applications:

            for vision_model in vision_models:
        
                initial_state = {
                    "image_path": image_path,
                    
                    "image_data": None,
                    
                    "extracted_data": {},
                    
                    "validation_result": {},
                    
                    "retry_count": 0,
                    
                    "messages": [],
                    
                    "result": {},
                    
                    "errors": [],
                    
                    "warnings": [],
                    
                    "processing_time": 0.0
                }
                
                
                final_state = app.invoke(initial_state)
                
                total_time = (datetime.now() - start_time).total_seconds()
                
                print(f"  EXTRACTION COMPLETE")
            
                print(f"   Total time: {total_time:.2f}s")
            
                print("="*60 + "\n")
                
                return final_state["result"]

output_file = f"license_extraction_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jsonl"
    
with open(output_file, "a") as f:

    for image_path, application in paths:
    
        result = extract_license_info(image_path)
        
        print("FINAL RESULTS")
        
        print("="*60)
        
        print(json.dumps(result, indent=2))
        
        json.dump(result, f)
    
print(f"\nResults saved to: {output_file}")

Image: https://raw.githubusercontent.com/agapebondservant/dla_poc/refs/heads/main/notebooks/data2/DENVER-25CAP-00000-04SVL-ID.jpeg
Started: 2025-11-22 07:06:35
✓ STEP 1: Image Loading & Preprocessing
✅ Image loaded: time: 0.00s
✓ STEP 2: AI-Powered Data Extraction
{'license_number': 'A123456', 'first_name': 'JOHN', 'middle_name': None, 'last_name': 'SMITH', 'date_of_birth': '01/01/1990', 'issue_date': '02/01/2020', 'expiration_date': '02/01/2025', 'street_address': '123 MAIN ST', 'city': 'DENVER', 'state': 'CO', 'zip_code': '80201', 'gender': 'M', 'height': '5\'10"', 'weight': '170 lbs', 'eye_color': 'BLU', 'hair_color': 'BLK', 'license_class': 'C', 'restrictions': None, 'endorsements': None, 'issuing_state': 'Colorado'}
✅ Extraction completed
   Fields extracted: 17/20
   Time: 0.86s

   Extracted data:
   --------------------------------------------------------
   license_number: A123456
   first_name: JOHN
   last_name: SMITH
   date_of_birth: 01/01/1990
   issue_date: 02/01/2020
  

,application_id,application_data,image_path
0,DENVER-25CAP-00000-04SUF-ID,"{'data': [{'balance': 0, 'booking': False, 'cr...",https://raw.githubusercontent.com/agapebondser...
1,DENVER-25CAP-00000-04SVL-ID,"{'data': [{'balance': 0, 'booking': False, 'cr...",https://raw.githubusercontent.com/agapebondser...
2,DENVER-25CAP-00000-04SWA-ID,"{'data': [{'balance': 0, 'booking': False, 'cr...",https://raw.githubusercontent.com/agapebondser...
3,DENVER-25CAP-00000-04SZC-ID,"{'data': [{'balance': 0, 'booking': False, 'cr...",https://raw.githubusercontent.com/agapebondser...
4,DENVER-25CAP-00000-04TEE-ID,"{'data': [{'balance': 0, 'booking': False, 'cr...",https://raw.githubusercontent.com/agapebondser...
5,DENVER-25CAP-00000-04TFN-ID,"{'data': [{'balance': 0, 'booking': False, 'cr...",https://raw.githubusercontent.com/agapebondser...
6,DENVER-25CAP-00000-04TIP-ID,"{'data': [{'balance': 0, 'booking': False, 'cr...",https://raw.githubusercontent.com/agapebondser...
7,DENVER-25CAP-00000-04TIV-ID,"{'data': [{'balance': 0, 'booking': False, 'cr...",https://raw.githubusercontent.com/agapebondser...
8,DENVER-25CAP-00000-04TJX-ID,"{'data': [{'balance': 0, 'booking': False, 'cr...",https://raw.githubusercontent.com/agapebondser...
9,DENVER-25CAP-00000-04TK7-ID,"{'data': [{'balance': 0, 'booking': False, 'cr...",https://raw.githubusercontent.com/agapebondser...


In [29]:
##############################################################################
# Register custom block
##############################################################################
from sdg_hub.core.blocks.base import BaseBlock
from sdg_hub.core.blocks.llm.llm_chat_block import LLMChatBlock
from sdg_hub.core.blocks.registry import BlockRegistry
import litellm
from pydantic import ConfigDict
import validators
from sdg_hub.core.utils.logger_config import setup_logger
from litellm import acompletion, completion

@BlockRegistry.register("CustomLLMMultimodalBlock", "llm", "Extension of BaseBlock that supports multimodal models")
class CustomLLMMultimodalBlock(LLMChatBlock):
    """Extends LLMChatBlock to support multimodal models"""

    model_config = ConfigDict(extra="allow")

    def monkey_patch_messages(self, records):
        """Adds <image_url> message to the list of existing messages"""

        for i, record in enumerate(records):
            
            image_url = None
    
            user = list(filter(lambda x: x["role"]=="user", record))[0]        
                
            _, _, image_url = user["content"].partition("```image_url: ")
        
            if not validators.url(image_url):
    
                raise ValueError(f"Error processing image_url: Ensure image_url={image_url} is valid")
                
            user["content"] = [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url,
                        "detail": "high",
                    },
                },
                {
                    "type": "text",
                    "text": "Extract the data from the image",
                },
            ]

        print(records)

        return records

    def _generate_sync(
        self,
        messages_list: list[list[dict[str, Any]]],
        completion_kwargs: dict[str, Any],
    ) -> list[list[dict[str, Any]]]:
        """Generate responses synchronously.

        Parameters
        ----------
        messages_list : list[list[dict[str, Any]]]
            List of message lists to process.
        completion_kwargs : dict[str, Any]
            Kwargs for LiteLLM completion.

        Returns
        -------
        list[list[dict[str, Any]]]
            List of response lists, each containing LiteLLM completion response dictionaries.
        """

        logger = setup_logger(__name__)
        
        responses = []

        messages_list = self.monkey_patch_messages(messages_list)

        for i, messages in enumerate(messages_list):
            try:
                response = completion(messages=messages, **completion_kwargs)
                # Extract response based on n parameter
                n_value = completion_kwargs.get("n", 1)
                if n_value > 1:
                    response_data = [
                        self._message_to_dict(choice.message)
                        for choice in response.choices
                    ]
                else:
                    response_data = [self._message_to_dict(response.choices[0].message)]
                responses.append(response_data)

                # Log progress for large batches
                if (i + 1) % 10 == 0:
                    logger.debug(
                        "Generated %d/%d responses",
                        i + 1,
                        len(messages_list),
                        extra={
                            "block_name": self.block_name,
                            "progress": f"{i + 1}/{len(messages_list)}",
                        },
                    )

            except Exception as e:
                logger.error(
                    "Failed to generate response for sample %d: %s",
                    i,
                    str(e),
                    extra={
                        "block_name": self.block_name,
                        "sample_index": i,
                        "error": str(e),
                    },
                )
                raise

        return responses

    async def _generate_async(
        self,
        messages_list: list[list[dict[str, Any]]],
        completion_kwargs: dict[str, Any],
        flow_max_concurrency: Optional[int] = None,
    ) -> list[list[dict[str, Any]]]:
        """Generate responses asynchronously.

        Parameters
        ----------
        messages_list : list[list[dict[str, Any]]]
            List of message lists to process.
        completion_kwargs : dict[str, Any]
            Kwargs for LiteLLM acompletion.
        flow_max_concurrency : Optional[int], optional
            Maximum concurrency for async requests.

        Returns
        -------
        list[list[dict[str, Any]]]
            List of response lists, each containing LiteLLM completion response dictionaries.
        """

        try:

            logger = setup_logger(__name__)

            messages_list = self.monkey_patch_messages(messages_list)
            
            if flow_max_concurrency is not None:
                # Validate max_concurrency parameter
                if flow_max_concurrency < 1:
                    raise ValueError(
                        f"max_concurrency must be greater than 0, got {flow_max_concurrency}"
                    )

                # Adjust concurrency based on n parameter (number of completions per request)
                effective_concurrency = flow_max_concurrency
                n_value = completion_kwargs.get("n", 1)

                if n_value and n_value > 1:
                    if flow_max_concurrency >= n_value:
                        # Adjust concurrency to account for n completions per request
                        effective_concurrency = flow_max_concurrency // n_value
                        logger.debug(
                            "Adjusted max_concurrency from %d to %d for n=%d completions per request",
                            flow_max_concurrency,
                            effective_concurrency,
                            n_value,
                            extra={
                                "block_name": self.block_name,
                                "original_max_concurrency": flow_max_concurrency,
                                "adjusted_max_concurrency": effective_concurrency,
                                "n_value": n_value,
                            },
                        )
                    else:
                        # Warn when max_concurrency is less than n
                        logger.warning(
                            "max_concurrency (%d) is less than n (%d). Consider increasing max_concurrency for optimal performance.",
                            flow_max_concurrency,
                            n_value,
                            extra={
                                "block_name": self.block_name,
                                "max_concurrency": flow_max_concurrency,
                                "n_value": n_value,
                            },
                        )
                        effective_concurrency = flow_max_concurrency

                # Use semaphore for concurrency control
                semaphore = asyncio.Semaphore(effective_concurrency)
                tasks = [
                    self._make_acompletion(messages, completion_kwargs, semaphore)
                    for messages in messages_list
                ]
            else:
                # No concurrency limit
                tasks = [
                    self._make_acompletion(messages, completion_kwargs)
                    for messages in messages_list
                ]

            responses = await asyncio.gather(*tasks)
            return responses

        except Exception as e:
            logger.error(
                "Failed to generate async responses: %s",
                str(e),
                extra={
                    "block_name": self.block_name,
                    "batch_size": len(messages_list),
                    "error": str(e),
                },
            )
            raise

        

In [30]:
from datasets import load_dataset, DatasetDict, Dataset
from sdg_hub.core.flow import FlowRegistry, Flow
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
from typing import Any, Optional
import asyncio
import nest_asyncio
nest_asyncio.apply()

items_adapter = TypeAdapter(list[LicenseApplication])

applications = utils.group_files_by_id("https://github.com/agapebondservant/dla_poc", "notebooks/data2")

df = pd.DataFrame(applications).drop('application_data', axis=1)

df["model_name"] = os.getenv("LLAMASCOUT4_LLM_NAME")

dataset = Dataset.from_pandas(df)

flow_path = "flows/drivers_license_validation/flow.yaml"

# columns_to_keep = ["code_id", "code", "markdown", "summary", "summary_type", "eval_summary_relevance", "eval_summary_faithfulness"]

flow = Flow.from_yaml(flow_path)

flow.set_model_config(
    model=os.getenv("LLAMASCOUT4_LLM_NAME"),
    api_base=os.getenv("LLAMASCOUT4_LLM_BASE"),
    api_key=os.getenv("LLAMASCOUT4_LLM_KEY"),
    temperature=0,
    max_tokens = 8192,
    response_format={"type": "json_object"},
    top_k=1,
)

converted_dataset = flow.generate(dataset, max_concurrency=10)

# converted_dataset.to_pandas()

converted_dataset.to_json("dataset_test.jsonl")

[23:17:53] INFO     Loading flow from: flows/drivers_license_validation/flow.yaml                       ]8;id=172068;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=138279;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py#140\140]8;;\

           INFO     Auto-detected 1 LLM blocks for configuration: ['extract_data_from_image']           ]8;id=223771;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=956488;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py#825\825]8;;\

           INFO     Successfully configured 1 LLM blocks with: model:                                   ]8;id=734752;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=200545;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py#867\867]8;;\
                    'openrouter/meta-llama/llama-4-scout', api_base: 'https://openrouter.ai/api/v1',               
                    api_key: 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,            
                    temperature: 0, max_tokens: 8192, response_format: {'type': 'json_object'}, top_k:             
                    1                                                                                              

           INFO     Configured blocks: ['extract_data_from_image']                                      ]8;id=709924;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=123206;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py#870\870]8;;\

           INFO     Converting datasets.Dataset to pd.DataFrame for processing                          ]8;id=682186;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=777857;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py#307\307]8;;\

...

[23:18:11] ERROR    Failed to generate async responses: litellm.BadRequestError:                   ]8;id=660512;file:///tmp/ipykernel_16863/238674134.py\238674134.py]8;;\:]8;id=80926;file:///tmp/ipykernel_16863/238674134.py#206\206]8;;\
                    OpenrouterException - {"error":{"message":"Unknown mimetype for image:                         
                    https://raw.githubusercontent.com/agapebondservant/dla_poc/refs/heads/main/not                 
                    ebooks/data2/DENVER-25CAP-00000-04TK7-ID.pdf","code":400,"metadata":{"provider                 
                    _name":"Google"}},"user_id":"user_33t3ge808GhkaZN83OCtmqq4BDr"} LiteLLM                        
                    Retried: 6 times                                                                               

[23:18:11] ERROR    Block 'extract_data_from_image' failed during execution: litellm.BadRequestError:   ]8;id=987755;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py\base.py]8;;\:]8;id=244454;file:///opt/app-root/lib64/python3.11/site-packages/sdg_hub/core/flow/base.py#714\714]8;;\
                    OpenrouterException - {"error":{"message":"Unknown mimetype for image:                         
                    https://raw.githubusercontent.com/agapebondservant/dla_poc/refs/heads/main/notebook            
                    s/data2/DENVER-25CAP-00000-04TK7-ID.pdf","code":400,"metadata":{"provider_name":"Go            
                    ogle"}},"user_id":"user_33t3ge808GhkaZN83OCtmqq4BDr"} LiteLLM Retried: 6 times                 

╭────────────────────────────────────── Drivers License Validation - Failed ──────────────────────────────────────╮
│                                        Flow Execution Summary                                                   │
│ ┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓           │
│ ┃ Block Name           ┃ Type            ┃   Duration ┃     Rows     ┃     Columns     ┃   Status   ┃           │
│ ┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩           │
│ │ extract_data_from_i… │ PromptBuilderB… │      0.00s │   10 → 10    │       +1        │     ✓      │           │
│ │ extract_data_from_i… │ CustomLLMMulti… │     17.30s │   10 → ❌    │        —        │     ✗      │           │
│ ├──────────────────────┼─────────────────┼────────────┼──────────────┼─────────────────┼────────────┤           │
│ │ TOTAL                │ 2 blocks        │     17.31s │   0 final    │     0 final     │    1/2     │           │
│ └──────────────────────┴─────────────────┴────────────┴──────────────┴─────────────────┴────────────┘           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

FlowValidationError: Block 'extract_data_from_image' execution failed: litellm.BadRequestError: OpenrouterException - {"error":{"message":"Unknown mimetype for image: https://raw.githubusercontent.com/agapebondservant/dla_poc/refs/heads/main/notebooks/data2/DENVER-25CAP-00000-04TK7-ID.pdf","code":400,"metadata":{"provider_name":"Google"}},"user_id":"user_33t3ge808GhkaZN83OCtmqq4BDr"} LiteLLM Retried: 6 times


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



In [32]:
from docling.document_converter import DocumentConverter
source = "https://raw.githubusercontent.com/agapebondservant/dla_poc/refs/heads/main/notebooks/data2/DENVER-25CAP-00000-04TJX-ID.pdf" # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.pictures)

2025-11-23 23:27:12,351 - INFO - Going to convert document batch...
2025-11-23 23:27:12,352 - INFO - Initializing pipeline for StandardPdfPipeline with options hash d291d1f79894f05d312cc90dd3fdf3d3
2025-11-23 23:27:12,352 - INFO - Accelerator device: 'cuda:0'
2025-11-23 23:27:13,736 - INFO - Accelerator device: 'cuda:0'
2025-11-23 23:27:14,345 - INFO - Accelerator device: 'cuda:0'
2025-11-23 23:27:14,878 - INFO - Processing document DENVER-25CAP-00000-04TJX-ID.pdf
2025-11-23 23:27:16,632 - INFO - Finished converting document DENVER-25CAP-00000-04TJX-ID.pdf in 4.74 sec.


[PictureItem(self_ref='#/pictures/0', parent=RefItem(cref='#/body'), children=[RefItem(cref='#/texts/0'), RefItem(cref='#/texts/1'), RefItem(cref='#/texts/2'), RefItem(cref='#/texts/3'), RefItem(cref='#/texts/4'), RefItem(cref='#/texts/5'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/7'), RefItem(cref='#/texts/8'), RefItem(cref='#/texts/9'), RefItem(cref='#/texts/10'), RefItem(cref='#/texts/11'), RefItem(cref='#/texts/12'), RefItem(cref='#/texts/13'), RefItem(cref='#/groups/1')], content_layer=<ContentLayer.BODY: 'body'>, label=<DocItemLabel.PICTURE: 'picture'>, prov=[ProvenanceItem(page_no=1, bbox=BoundingBox(l=11.91973876953125, t=784.4516377449036, r=250.40283203125, b=630.186767578125, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>), charspan=(0, 0))], captions=[], references=[], footnotes=[], image=None, annotations=[]), PictureItem(self_ref='#/pictures/1', parent=RefItem(cref='#/body'), children=[RefItem(cref='#/texts/15'), RefItem(cref='#/texts/16'), RefItem(cref='#/t

In [41]:
import logging
import time
from pathlib import Path

from docling_core.types.doc import ImageRefMode, PictureItem, TableItem

from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

_log = logging.getLogger(__name__)

IMAGE_RESOLUTION_SCALE = 2.0


logging.basicConfig(level=logging.INFO)

input_doc_path = "https://raw.githubusercontent.com/agapebondservant/dla_poc/refs/heads/main/notebooks/data2/DENVER-25CAP-00000-04TJX-ID.pdf"
output_dir = "scratch"

pipeline_options = PdfPipelineOptions()
pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
pipeline_options.generate_page_images = True
pipeline_options.generate_picture_images = True

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

start_time = time.time()

conv_res = doc_converter.convert(input_doc_path)

os.makedirs(output_dir, exist_ok=True)
doc_filename = conv_res.input.file.stem


for page_no, page in conv_res.document.pages.items():
    page_no = page.page_no
    page_image_filename = f"{output_dir}/{doc_filename}-{page_no}.png"
    with open(page_image_filename, "wb") as fp:
        page.image.pil_image.save(fp, format="PNG")
    break

2025-11-23 23:42:27,798 - INFO - Going to convert document batch...
2025-11-23 23:42:27,799 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 85ac70aecbb3e9d18a585ddbe6342108
2025-11-23 23:42:27,800 - INFO - Accelerator device: 'cuda:0'
2025-11-23 23:42:29,169 - INFO - Accelerator device: 'cuda:0'
2025-11-23 23:42:29,788 - INFO - Accelerator device: 'cuda:0'
2025-11-23 23:42:30,273 - INFO - Processing document DENVER-25CAP-00000-04TJX-ID.pdf
2025-11-23 23:42:32,441 - INFO - Finished converting document DENVER-25CAP-00000-04TJX-ID.pdf in 4.75 sec.
